Name :- Heet Dhanuka

Batch :- B2

Roll No. :- 34

DL practical 6b

------

Aim :- Design and implement a CNN model for fruit classification, and then apply transfer learning by utilizing at least  two pretrained model like VGG19 and RESNET 50 to achieve the same objective.  

In [ ]:
!pip install kaggle

# Download the dataset (replace with your Kaggle API token)
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download utkarshsaxenadn/fruits-classification

# Code to unzip the file
import zipfile
with zipfile.ZipFile('/content/fruits-classification.zip', 'r') as zip_ref:
  zip_ref.extractall('/content')


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2


# Define the image dimensions
img_width, img_height = 224, 224
train_data_dir = '/content/Fruits Classification/train'
validation_data_dir = '/content/Fruits Classification/test'
nb_train_samples = 41322
nb_validation_samples = 13877
epochs = 1
batch_size = 16

import os

num_classes = len(os.listdir(train_data_dir))
print(f"Number of classes: {num_classes}")

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/utkarshsaxenadn/fruits-classification
License(s): CC0-1.0
 85% 72.0M/84.8M [00:00<00:00, 202MB/s]
100% 84.8M/84.8M [00:00<00:00, 165MB/s]
Number of classes: 5


In [ ]:
# CNN Model
def create_cnn_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes , activation='softmax'))  # Assuming 131 classes
  return model

In [ ]:
# Transfer Learning with VGG19
def create_vgg19_model():
  base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu')(x)
  predictions = Dense(num_classes, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)

  for layer in base_model.layers:
    layer.trainable = False

  return model

In [ ]:
# Transfer Learning with ResNet50
def create_resnet50_model():
  base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu')(x)
  predictions = Dense(num_classes, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)

  for layer in base_model.layers:
    layer.trainable = False

  return model

In [ ]:
# Data Augmentation and Generators
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

training_set = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_set = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 9700 images belonging to 5 classes.
Found 100 images belonging to 5 classes.


In [ ]:
# Train the models
def train_model(model, model_name):
  model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
  model.fit(
      training_set,
      steps_per_epoch=nb_train_samples // batch_size,
      epochs=epochs,
      validation_data=test_set,
      validation_steps=nb_validation_samples // batch_size)

  model.save(f"{model_name}.h5")
  print(f"{model_name} model trained and saved.")


# Create and train CNN
cnn_model = create_cnn_model()
train_model(cnn_model, "cnn_model")

# Create and train VGG19
vgg19_model = create_vgg19_model()
train_model(vgg19_model, "vgg19_model")

# Create and train ResNet50
resnet50_model = create_resnet50_model()
train_model(resnet50_model, "resnet50_model")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 607/2582 ━━━━━━━━━━━━━━━━━━━━ 7:24 225ms/step - accuracy: 0.3502 - loss: 1.4531

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2582/2582 ━━━━━━━━━━━━━━━━━━━━ 147s 54ms/step - accuracy: 0.4065 - loss: 1.3584 - val_accuracy: 0.5300 - val_loss: 1.1498


cnn_model model trained and saved.
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
2582/2582 ━━━━━━━━━━━━━━━━━━━━ 160s 59ms/step - accuracy: 0.5110 - loss: 1.2889 - val_accuracy: 0.6200 - val_loss: 0.9830


vgg19_model model trained and saved.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
2582/2582 ━━━━━━━━━━━━━━━━━━━━ 156s 55ms/step - accuracy: 0.2567 - loss: 1.5953 - val_accuracy: 0.3200 - val_loss: 1.5798


resnet50_model model trained and saved.
